# Notebook 01: Data Wrangling for C. elegans Motility Analysis

This notebook handles:
- Loading raw WormLab data (Position, Fit, CurvatureMap files)
- Reading metadata from YAML files
- Processing track data and calculating metrics
- Normalizing track positions to origin (0,0)
- Calculating instantaneous speeds
- Computing movement metrics (straightness index, path length, etc.)
- Extracting thrashing frequency from curvature data
- Saving processed data for downstream analysis

## 1. Import Libraries

In [8]:
import pandas as pd
import numpy as np
import yaml
from pathlib import Path
import matplotlib.pyplot as plt
from scipy import signal
from scipy.signal import find_peaks
import warnings
warnings.filterwarnings('ignore')

print("Libraries imported successfully!")

Libraries imported successfully!


## 2. Configuration and Path Setup

In [9]:
# Base paths
BASE_DIR = Path(r'C:\Users\MBF\Motility_analysis')
RAW_DATA_DIR = BASE_DIR / 'Data' / 'Raw'
PROCESSED_DATA_DIR = BASE_DIR / 'Data' / 'Processed'
RESULTS_DIR = BASE_DIR / 'results'

# Create output directories if they don't exist
PROCESSED_DATA_DIR.mkdir(parents=True, exist_ok=True)
RESULTS_DIR.mkdir(parents=True, exist_ok=True)

print(f"Base directory: {BASE_DIR}")
print(f"Raw data directory: {RAW_DATA_DIR}")
print(f"Processed data directory: {PROCESSED_DATA_DIR}")
print(f"Results directory: {RESULTS_DIR}")

Base directory: C:\Users\MBF\Motility_analysis
Raw data directory: C:\Users\MBF\Motility_analysis\Data\Raw
Processed data directory: C:\Users\MBF\Motility_analysis\Data\Processed
Results directory: C:\Users\MBF\Motility_analysis\results


## 3. Helper Functions

In [10]:
def load_metadata(yaml_path):
    """
    Load metadata from YAML file
    """
    with open(yaml_path, 'r') as f:
        metadata = yaml.safe_load(f)
    return metadata


def load_position_file(position_path):
    """
    Load Position CSV file, skipping first row (track labels) and using second row as header
    Returns: DataFrame with Frame, Time, and X/Y coordinate pairs for each track
    """
    df = pd.read_csv(position_path, skiprows=1)
    return df


def extract_track_data(position_df):
    """
    Extract individual track data from Position DataFrame
    Returns: Dictionary of DataFrames, one per track
    """
    tracks = {}
    
    # Get column names excluding Frame and Time
    columns = position_df.columns.tolist()
    
    # Extract track numbers from column names (format: "1 x", "1 y", "2 x", "2 y", etc.)
    track_nums = set()
    for col in columns:
        if col not in ['Frame', 'Time']:
            # Extract track number from column name
            track_num = col.split()[0]
            track_nums.add(track_num)
    
    # Create DataFrame for each track
    for track_num in sorted(track_nums):
        x_col = f"{track_num} x"
        y_col = f"{track_num} y"
        
        if x_col in columns and y_col in columns:
            track_df = pd.DataFrame({
                'Frame': position_df['Frame'],
                'Time': position_df['Time'],
                'X': position_df[x_col],
                'Y': position_df[y_col]
            })
            
            # Remove NaN values
            track_df = track_df.dropna()
            
            if len(track_df) > 0:
                tracks[track_num] = track_df
    
    return tracks


def normalize_track_to_origin(track_df):
    """
    Normalize track so it starts at (0, 0)
    """
    track_normalized = track_df.copy()
    track_normalized['X'] = track_df['X'] - track_df['X'].iloc[0]
    track_normalized['Y'] = track_df['Y'] - track_df['Y'].iloc[0]
    return track_normalized


def calculate_speed(track_df):
    """
    Calculate instantaneous speed from position data
    Speed in microns per second
    """
    track_df = track_df.copy()
    
    # Calculate displacement
    dx = track_df['X'].diff()
    dy = track_df['Y'].diff()
    dt = track_df['Time'].diff()
    
    # Calculate distance and speed
    distance = np.sqrt(dx**2 + dy**2)
    speed = distance / dt
    
    track_df['Speed'] = speed
    
    return track_df


def calculate_path_metrics(track_df):
    """
    Calculate various path metrics:
    - Total path length
    - Displacement (straight-line distance from start to end)
    - Straightness index (displacement / path length)
    - Mean speed
    """
    # Calculate distances between consecutive points
    dx = track_df['X'].diff()
    dy = track_df['Y'].diff()
    distances = np.sqrt(dx**2 + dy**2)
    
    # Total path length
    path_length = distances.sum()
    
    # Displacement (start to end)
    displacement = np.sqrt(
        (track_df['X'].iloc[-1] - track_df['X'].iloc[0])**2 +
        (track_df['Y'].iloc[-1] - track_df['Y'].iloc[0])**2
    )
    
    # Straightness index
    straightness = displacement / path_length if path_length > 0 else 0
    
    # Mean speed (excluding NaN values)
    if 'Speed' in track_df.columns:
        mean_speed = track_df['Speed'].dropna().mean()
    else:
        mean_speed = np.nan
    
    return {
        'path_length': path_length,
        'displacement': displacement,
        'straightness': straightness,
        'mean_speed': mean_speed
    }


def calculate_fatigue_index(track_df, window_size=50):
    """
    Calculate fatigue index as ratio of speed in last window vs first window
    Values < 1 indicate slowing down (fatigue)
    """
    if 'Speed' not in track_df.columns or len(track_df) < 2 * window_size:
        return np.nan
    
    first_window_speed = track_df['Speed'].iloc[:window_size].mean()
    last_window_speed = track_df['Speed'].iloc[-window_size:].mean()
    
    if first_window_speed > 0:
        fatigue_index = last_window_speed / first_window_speed
    else:
        fatigue_index = np.nan
    
    return fatigue_index


def load_curvature_file(curvature_path):
    """
    Load CurvatureMap CSV file
    """
    df = pd.read_csv(curvature_path, skiprows=1)
    return df


def calculate_thrashing_frequency(curvature_df, midbody_column=None):
    """
    Calculate thrashing frequency from curvature data
    Uses midbody curvature oscillations to detect thrashing
    Returns frequency in Hz
    """
    # If no specific column specified, use middle column (midbody)
    curvature_columns = [col for col in curvature_df.columns if col not in ['Frame', 'Time']]
    
    if len(curvature_columns) == 0:
        return np.nan
    
    if midbody_column is None:
        # Use middle column as midbody
        midbody_column = curvature_columns[len(curvature_columns) // 2]
    
    # Extract midbody curvature
    curvature = curvature_df[midbody_column].dropna()
    time = curvature_df['Time'].loc[curvature.index]
    
    if len(curvature) < 10:
        return np.nan
    
    # Find peaks in curvature (thrashing events)
    # Use prominence to identify significant oscillations
    peaks, properties = find_peaks(curvature, prominence=50)
    
    if len(peaks) < 2:
        return 0.0
    
    # Calculate frequency
    total_time = time.iloc[-1] - time.iloc[0]
    frequency = len(peaks) / total_time  # Hz
    
    return frequency


print("Helper functions defined successfully!")

Helper functions defined successfully!


## 4. Discover Data Structure

In [14]:
# Find all dates (subdirectories in Raw)
dates = [d for d in RAW_DATA_DIR.iterdir() if d.is_dir()]

print(f"Found {len(dates)} date folder(s):")
for date_dir in dates:
    print(f"  - {date_dir.name}")

# Find all genotypes across all dates
all_genotypes = []
data_structure = {}

for date_dir in dates:
    wormlab_dir = date_dir / 'Wormlab_processed'
    if wormlab_dir.exists():
        genotype_roots = [g for g in wormlab_dir.iterdir() if g.is_dir()]
    else:
        genotype_roots = [g for g in date_dir.iterdir() if g.is_dir()]
        print(f"  NOTE: {date_dir.name} has no Wormlab_processed folder; using direct subfolders.")

    data_structure[date_dir.name] = {}

    for genotype_dir in genotype_roots:
        genotype_name = genotype_dir.name
        all_genotypes.append(genotype_name)

        # Count videos for this genotype
        videos = [v for v in genotype_dir.iterdir() if v.is_dir() and v.name.startswith('000')]
        data_structure[date_dir.name][genotype_name] = len(videos)

        print(f"  {date_dir.name}/{genotype_name}: {len(videos)} video(s)")

unique_genotypes = sorted(set(all_genotypes))
print(f"\nUnique genotypes found: {unique_genotypes}")

Found 1 date folder(s):
  - 260224
  NOTE: 260224 has no Wormlab_processed folder; using direct subfolders.
  260224/OGxxxx_1mMaux: 4 video(s)
  260224/OGxxxx_noaux: 4 video(s)

Unique genotypes found: ['OGxxxx_1mMaux', 'OGxxxx_noaux']


## 5. Load and Process All Data

In [18]:
# Master data storage
all_track_data = []  # List of dictionaries with all track info
all_track_coordinates = []  # For XY plotting
all_metrics = []  # Summary metrics per track
all_thrashing = []  # Thrashing frequency data

# Process each date
for date_dir in dates:
    date_name = date_dir.name
    wormlab_dir = date_dir / 'Wormlab_processed'

    if wormlab_dir.exists():
        genotype_roots = [g for g in wormlab_dir.iterdir() if g.is_dir()]
    else:
        genotype_roots = [g for g in date_dir.iterdir() if g.is_dir()]
        print(f"\nNOTE: {date_name} has no Wormlab_processed folder; using direct subfolders.")

    # Process each genotype
    for genotype_dir in genotype_roots:
        genotype_name = genotype_dir.name
        print(f"\nProcessing {date_name}/{genotype_name}...")

        # Load metadata if available
        metadata_files = list(genotype_dir.glob('metadata_*.yaml'))
        metadata = {}
        if metadata_files:
            metadata = load_metadata(metadata_files[0])
            print(f"  Loaded metadata: {metadata.get('strain', 'N/A')}, genotype: {metadata.get('genotype', 'N/A')}")
        auxin_mM = metadata.get('auxin_mM', np.nan)

        # Process each video
        videos = [v for v in genotype_dir.iterdir() if v.is_dir() and v.name.startswith('000')]
        if not videos:
            print(f"  WARNING: No video folders (000*) found in {genotype_dir}")
            continue

        for video_dir in sorted(videos):
            video_id = video_dir.name
            print(f"  Processing video {video_id}...")

            # Find Position file (may be Position.csv or Position-1.csv, etc.)
            position_files = list(video_dir.glob('Position*.csv'))
            if not position_files:
                print(f"    WARNING: No Position file found in {video_dir}")
                continue

            position_file = position_files[0]

            try:
                # Load position data
                position_df = load_position_file(position_file)

                # Extract individual tracks
                tracks = extract_track_data(position_df)
                print(f"    Found {len(tracks)} track(s)")

                # Process each track
                for track_num, track_df in tracks.items():
                    # Calculate speed
                    track_df = calculate_speed(track_df)

                    # Normalize to origin
                    track_normalized = normalize_track_to_origin(track_df)

                    # Calculate metrics
                    metrics = calculate_path_metrics(track_df)
                    fatigue = calculate_fatigue_index(track_df)

                    # Store track data
                    track_info = {
                        'date': date_name,
                        'genotype': genotype_name,
                        'strain': metadata.get('strain', genotype_name),
                        'auxin_mM': auxin_mM,
                        'video_id': video_id,
                        'track_num': track_num,
                        'track_id': f"{genotype_name}_{video_id}_{track_num}",
                        'mean_speed': metrics['mean_speed'],
                        'path_length': metrics['path_length'],
                        'displacement': metrics['displacement'],
                        'straightness': metrics['straightness'],
                        'fatigue_index': fatigue,
                        'n_frames': len(track_df),
                        'duration': track_df['Time'].iloc[-1] - track_df['Time'].iloc[0]
                    }

                    all_metrics.append(track_info)

                    # Store normalized coordinates for plotting
                    track_normalized['genotype'] = genotype_name
                    track_normalized['auxin_mM'] = auxin_mM
                    track_normalized['track_id'] = track_info['track_id']
                    all_track_coordinates.append(track_normalized)

                    # Process curvature data for thrashing
                    curvature_files = list(video_dir.glob(f'CurvatureMap*Track_{track_num}.csv'))
                    if curvature_files:
                        try:
                            curvature_df = load_curvature_file(curvature_files[0])
                            thrashing_freq = calculate_thrashing_frequency(curvature_df)

                            thrashing_info = {
                                'date': date_name,
                                'genotype': genotype_name,
                                'strain': metadata.get('strain', genotype_name),
                                'auxin_mM': auxin_mM,
                                'video_id': video_id,
                                'track_num': track_num,
                                'track_id': track_info['track_id'],
                                'thrashing_frequency_hz': thrashing_freq
                            }

                            all_thrashing.append(thrashing_info)
                        except Exception as e:
                            print(f"      WARNING: Error processing curvature for track {track_num}: {e}")

            except Exception as e:
                print(f"    ERROR processing {position_file}: {e}")
                continue

print(f"\n\nDATA PROCESSING COMPLETE!")
print(f"Total tracks processed: {len(all_metrics)}")
print(f"Tracks with thrashing data: {len(all_thrashing)}")


NOTE: 260224 has no Wormlab_processed folder; using direct subfolders.

Processing 260224/OGxxxx_1mMaux...
  Processing video 0001...
    Found 10 track(s)
  Processing video 0002...
    Found 5 track(s)
  Processing video 0003...
    Found 5 track(s)
  Processing video 0004...
    Found 9 track(s)

Processing 260224/OGxxxx_noaux...
  Processing video 0001...
    Found 6 track(s)
  Processing video 0002...
    Found 6 track(s)
  Processing video 0003...
    Found 11 track(s)
  Processing video 0004...
    Found 12 track(s)


DATA PROCESSING COMPLETE!
Total tracks processed: 64
Tracks with thrashing data: 64


## 6. Create Summary DataFrames

In [16]:
# Convert to DataFrames
df_metrics = pd.DataFrame(all_metrics)
df_thrashing = pd.DataFrame(all_thrashing)
df_tracks = pd.concat(all_track_coordinates, ignore_index=True) if all_track_coordinates else pd.DataFrame()

# Display summary statistics
print("\n=== METRICS SUMMARY ===")
print(df_metrics.head())
print(f"\nShape: {df_metrics.shape}")

print("\n=== THRASHING SUMMARY ===")
print(df_thrashing.head())
print(f"\nShape: {df_thrashing.shape}")

print("\n=== TRACK COORDINATES SUMMARY ===")
print(df_tracks.head())
print(f"\nShape: {df_tracks.shape}")

# Summary by genotype
print("\n=== SUMMARY BY GENOTYPE ===")
summary_by_genotype = df_metrics.groupby('genotype').agg({
    'mean_speed': ['mean', 'std', 'count'],
    'straightness': ['mean', 'std'],
    'fatigue_index': ['mean', 'std']
})
print(summary_by_genotype)


=== METRICS SUMMARY ===
     date       genotype         strain video_id track_num  \
0  260224  OGxxxx_1mMaux  OGxxxx_1mMaux     0001         1   
1  260224  OGxxxx_1mMaux  OGxxxx_1mMaux     0001        10   
2  260224  OGxxxx_1mMaux  OGxxxx_1mMaux     0001         2   
3  260224  OGxxxx_1mMaux  OGxxxx_1mMaux     0001         3   
4  260224  OGxxxx_1mMaux  OGxxxx_1mMaux     0001         4   

                track_id  mean_speed   path_length  displacement  \
0   OGxxxx_1mMaux_0001_1  473.304367  14165.323551   4110.763155   
1  OGxxxx_1mMaux_0001_10  744.466036   1500.690950    686.124115   
2   OGxxxx_1mMaux_0001_2  465.642803  13985.680807   2055.968505   
3   OGxxxx_1mMaux_0001_3  455.841449  13642.683369   1966.355430   
4   OGxxxx_1mMaux_0001_4  476.374636  14257.212325    210.681784   

   straightness  fatigue_index  n_frames   duration  
0      0.290199       0.948489       420  29.928571  
1      0.457205            NaN        27   1.928571  
2      0.147005       1.018432 

## 7. Save Processed Data

In [17]:
# Save to CSV files
df_metrics.to_csv(PROCESSED_DATA_DIR / 'track_metrics.csv', index=False)
df_thrashing.to_csv(PROCESSED_DATA_DIR / 'thrashing_data.csv', index=False)
df_tracks.to_csv(PROCESSED_DATA_DIR / 'normalized_tracks.csv', index=False)

print("Processed data saved to:")
print(f"  - {PROCESSED_DATA_DIR / 'track_metrics.csv'}")
print(f"  - {PROCESSED_DATA_DIR / 'thrashing_data.csv'}")
print(f"  - {PROCESSED_DATA_DIR / 'normalized_tracks.csv'}")

print("\n✓ DATA WRANGLING COMPLETE!")

Processed data saved to:
  - C:\Users\MBF\Motility_analysis\Data\Processed\track_metrics.csv
  - C:\Users\MBF\Motility_analysis\Data\Processed\thrashing_data.csv
  - C:\Users\MBF\Motility_analysis\Data\Processed\normalized_tracks.csv

✓ DATA WRANGLING COMPLETE!
